# 参数管理

- 访问参数，用于调试，诊断和可视化
- 参数初始化
- 在不同模型组件间共享参数

In [1]:
'''具有隐藏层的多层感知机'''
import torch
from torch import nn
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2606],
        [-0.2089]], grad_fn=<AddmmBackward0>)

## 1. 参数访问

当通过sequential类定义模型时，可以通过索引访问模型任意层。每层的参数都在属性中

In [2]:
'''检查第二个全连接层的参数'''
print(net[2].state_dict())
print(net[2].weight)
print(type(net[2].weight.data)) # weights中有两个属性，可以通过data访问存有权重的张量

OrderedDict({'weight': tensor([[-0.2827, -0.2140,  0.3088, -0.2871,  0.1935, -0.3340, -0.0665,  0.1525]]), 'bias': tensor([-0.0847])})
Parameter containing:
tensor([[-0.2827, -0.2140,  0.3088, -0.2871,  0.1935, -0.3340, -0.0665,  0.1525]],
       requires_grad=True)
<class 'torch.Tensor'>


### 1.1 目标参数

参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [3]:
print(type(net[2].bias)) # 符合对象，包含值，梯度
print(net[2].bias) # 符合对象，包含值，梯度
print(net[2].bias.data) # 访问存有权重值的张量

net[2].weight == None # 还没有进行反向传播，所以目前还有梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0847], requires_grad=True)
tensor([-0.0847])


False

### 1.2 一次性访问所有参数

In [4]:
'''访问全连接层的参数，与，访问所有层的参数'''
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) # 使用*号将列表元素解包为单独的参数传递给print函数
print(*[(name, param.shape) for name, param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [5]:
'''另一种访问网络参数的方法'''
net.state_dict()['2.bias'].data # 访问2层偏置值

tensor([-0.0847])

### 1.3 从嵌套块中收集参数

In [11]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
        )
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1719],
        [-0.1719]], grad_fn=<AddmmBackward0>)

查看网络是如何工作的

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


由于时分层嵌套的，所以我们也可以通过嵌套列表索引访问

In [14]:
rgnet[0][1][0].bias.data

tensor([ 0.2589,  0.3112, -0.4578,  0.3674, -0.0416,  0.1366,  0.0804, -0.0237])

## 2. 参数初始化

### 2.1 内置初始化

In [15]:
'''将所有权重初始化为标准差为0.01的高斯随机变量，将偏置值设置为0'''
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0039,  0.0096, -0.0183, -0.0021]), tensor(0.))

In [16]:
'''将所有参数初始化为给定的常数，如初始化为1'''
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [17]:
'''对不同的块用不同的初试方法
下面使用Xavier初始化第一个神经网络层，然后将第三个神经网络初始化为常量42
'''
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)
        

tensor([-0.3226,  0.0368, -0.0309,  0.4126])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 2.2 自定义初始化

如下权重参数$w$的初始化
$$w \sim 
\begin{cases} 
U(5, 10) & \text{with probability } \frac{1}{4} \\
0 & \text{with probability } \frac{1}{2} \\
U(-10, -5) & \text{with probability } \frac{1}{4}
\end{cases}
$$


In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-6.5076, -8.2094,  0.0000,  9.4585],
        [-6.2986,  6.0288, -5.5219, -0.0000]], grad_fn=<SliceBackward0>)

In [19]:
'''始终可以直接设置参数'''
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.2094,  1.0000, 10.4585])

## 3. 参数绑定

有时希望在多个层间共享参数，定义一个稠密层，使用它的参数来设置另一层的参数

In [21]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net)

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)
